In [1]:
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go
pyo.init_notebook_mode(connected=True)
pd.set_option("display.max_columns",500)
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv("https://voteview.com/static/data/out/members/HSall_members.csv")

In [3]:
congresses = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_Congresses")[0]
congresses.columns = congresses.iloc[0]
congresses.drop(0,inplace=True)

In [4]:
congresses = congresses[congresses['Congress'].str.contains('Congress')]
congresses = congresses.query('Congress!="Congress"')
congresses['congress_no'] = congresses['Congress'].str.extract(r'(\d+)').astype(np.int64)

In [5]:
#Exact Democrat & Repulican only in house only
dfc = df[df['party_code'].isin([100,200])]
dfc = dfc[dfc['chamber']=="House"]
dfc = dfc[['congress','party_code','nominate_dim1']]
dfc = pd.merge(dfc,congresses[['Congress began','Congress ended','congress_no']],left_on='congress',right_on='congress_no')

In [6]:
party = {100:'Democrats',200:'Republican'}
dfc['party'] = dfc['party_code'].replace(party)
dfc['bin'] = pd.cut(dfc['nominate_dim1'],np.linspace(-1,1,21),labels = np.linspace(-1,0.9,20))
dfc['bin'] = round(dfc['bin'],1)

In [7]:
curr = dfc[dfc['congress']==115]

In [8]:
curr.head()

,congress,party_code,nominate_dim1,Congress began,Congress ended,congress_no,party,bin
32996,115,200,0.338,"January 3, 2017","January 3, 2019",115,Republican,0.3
32997,115,100,-0.390,"January 3, 2017","January 3, 2019",115,Democrats,-0.4
32998,115,200,0.367,"January 3, 2017","January 3, 2019",115,Republican,0.3
32999,115,200,0.607,"January 3, 2017","January 3, 2019",115,Republican,0.6
33000,115,200,0.543,"January 3, 2017","January 3, 2019",115,Republican,0.5


In [9]:
dem = curr[curr['party_code']==100]
rep = curr[curr['party_code']==200]
repdata = go.Scatter(
    x=rep['bin'].value_counts().sort_index().index,
    y = rep['bin'].value_counts().sort_index(),
        fill='tozeroy',name='rep')
demdata = go.Scatter(
    x=dem['bin'].value_counts().sort_index().index,
    y=dem['bin'].value_counts().sort_index(),        
    fill='tozeroy',name='dem')
data = [repdata,demdata]
pyo.iplot(data)

In [10]:
repdata = go.Histogram(x=rep['nominate_dim1'],marker=dict(color='red'),name='Republican')
demdata = go.Histogram(x=dem['nominate_dim1'],marker=dict(color='blue'),name='Democrat')
data = [repdata,demdata]
pyo.iplot(data)

In [16]:
data = []
layout = []

def adddata(congress,steps):
    curr = dfc[dfc['congress']==congress]
    dem = curr[curr['party_code']==100]

    rep = curr[curr['party_code']==200]

    yaxis='y{}'.format(steps)
    if steps == 1:
        repdata = go.Scatter(
            x=rep['bin'].value_counts().sort_index().index,
            y = rep['bin'].value_counts().sort_index(),
                fill='tozeroy',
            line=dict(color='red',shape='spline',smoothing=1.3),
            mode='lines'
   )
        demdata = go.Scatter(
            x=dem['bin'].value_counts().sort_index().index,
            y=dem['bin'].value_counts().sort_index(),        
            fill='tozeroy',
                        line=dict(color='blue',shape='spline',smoothing=1.3),
                        mode='lines'
                )
        return repdata, demdata
    else: 
        repdata = go.Scatter(
            x=rep['bin'].value_counts().sort_index().index,
            y = rep['bin'].value_counts().sort_index(),
                fill='tozeroy',
                yaxis='y{}'.format(steps),
                                line=dict(color='red',shape='spline',smoothing=1.3),
                        mode='lines')
        
        demdata = go.Scatter(
            x=dem['bin'].value_counts().sort_index().index,
            y=dem['bin'].value_counts().sort_index(),        
            fill='tozeroy',
                        yaxis='y{}'.format(steps),
                                line=dict(color='blue',shape='spline',smoothing=1.3),
                        mode='lines')
        return repdata, demdata
    

data = []
toplot = list(range(90,116))
steps = 1
start = min(toplot)
end = max(toplot)
step = 1/(end - start+1)
annotations = []
years = pd.to_datetime(dfc[dfc['congress']>=start]['Congress began'].unique()).year

layout = go.Layout(width=800,height=1200,showlegend=False,
                   xaxis=dict(range=[-1,1],
                              showgrid=False,
                              showticklabels=False),
                   title='Changing Extremities of Democrat & Republicans in recent congresses',
                   paper_bgcolor='rgb(233,233,233)',
                  plot_bgcolor='rgb(233,233,233)')



annotations.append( 
dict(
x=-0.01,
y=1.02,
showarrow=False,
text="Congress Start Year",
xref='paper',
yref='paper',
yanchor='middle',
xanchor='center'
        )
                   
                   
)

annotations.append( 
dict(
x=0.25,
y=-0.02,
showarrow=False,
text="<b>Democrats</b>",
xref='paper',
yref='paper',
yanchor='middle',
xanchor='center'
        )
                   
                   
)

annotations.append( 
dict(
x=0.75,
y=-0.02,
showarrow=False,
text="<b>Republicans</b>",
xref='paper',
yref='paper',
yanchor='middle',
xanchor='center'
        )
                   
                   
)
    
                          
                          
            
for j in toplot:
    for i in adddata(j,steps):
        data.append(i)
    if steps == 1:
        layout['yaxis'] = dict(domain=[1-step, 1], 
                               showgrid=False,
                               zeroline=False,range=[0,70],
                               showline=False,
                               showticklabels=False)
        print(j)
        annotations.append( 
                dict(
                x=-0.01,
                y=0.9807692307692308,
                showarrow=False,
                text=str(years[steps-1]),
                xref='paper',
                yref='paper',
                yanchor='middle',xanchor='center'
        ))
    else:
        axisno = 'yaxis' + str(j-start+1)
        layout[axisno] = dict(domain=[1-((j-start+1)*step),
                                      1-((j-start)*step)],
                              showgrid=False,
                              zeroline=False,
                              range=[0,70],
                              showline=False,
                              showticklabels=False)
        annotations.append( 
                dict(
                x=-0.01,
                y=(layout[axisno]['domain'][1]+ layout[axisno]['domain'][0])/2 ,
                showarrow=False,
                text=str(years[steps-1]),
                xref='paper',
                yref='paper',
                yanchor='middle',
                xanchor='center'))
        
        
        
        

    steps += 1
    
    
layout['annotations'] = annotations
fig = dict(data=data,layout=layout)
pyo.iplot(fig)

90


#Final to do:

- Manually add legend for Democrats
- Add explanatory labels